In [6]:
from langchain.vectorstores.faiss import FAISS
from langchain_core.documents import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chat_models.gigachat import GigaChat
from langchain.chains import create_retrieval_chain
from langchain_gigachat.chat_models import GigaChat
from langchain_gigachat.embeddings.gigachat import GigaChatEmbeddings
from pathlib import Path
from vectorization.v_a_c import *

In [7]:
API_KEY = "YjllY2FhYjgtNGRlMC00MDA4LWIwZmYtNjdlNjY0ZmI5OTc4OmRkMjZhOWFjLThhNTctNGM3ZC1iZjFkLWQ3NGY1NmRjNTQzMQ=="
CODE_DIR = Path("..\\data\\code")
DOC_DIR = Path("..\\data\\documentation")
VECTOR_STORE_PATH = "langchain_vector_store"

In [8]:
embedding=GigaChatEmbeddings(
        credentials=API_KEY,
        scope="GIGACHAT_API_PERS",
        verify_ssl_certs=False,
)

In [9]:
embeddings = GigaChatEmbeddings(
    credentials=API_KEY,
    verify_ssl_certs=False
)

system = SmartCodeDocSystem(embeddings, chunk_size=600, chunk_overlap=100)


In [10]:
documents = system.load_and_process_files(CODE_DIR, DOC_DIR)

system.create_vector_store(VECTOR_STORE_PATH)

Загрузка и обработка файлов...
Обработка кода из ..\data\code
  Обработано 10/19 файлов кода
Обработка документации из ..\data\documentation
  Обработано 5/17 файлов документации
  Обработано 10/17 файлов документации
  Обработано 15/17 файлов документации
Всего создано 406 чанков
  - Код: 98
  - Документация: 308
Создание векторного хранилища...
Сохранение в langchain_vector_store
Векторное хранилище создано и сохранено


In [11]:
system.load_vector_store(VECTOR_STORE_PATH)

Загрузка векторного хранилища из langchain_vector_store
Векторное хранилище загружено


True

In [12]:
llm = GigaChat(
    credentials=API_KEY,
    verify_ssl_certs=False,
)

smart_retriever = SmartRetriever(smart_system=system, k=3)

prompt = create_smart_prompt()
document_chain = create_stuff_documents_chain(llm=llm, prompt=prompt)
retrieval_chain = create_retrieval_chain(smart_retriever, document_chain)

In [13]:
query = "How to use FAISS vectorstore in LangChain?"

try:
    # Получаем ответ
    response = retrieval_chain.invoke({"input": query})
    
    # Выводим результат
    print("ОТВЕТ:")
    print(response["answer"])
    
    # Показываем информацию о поиске
    search_result = system.smart_search(query, k=3)
    print(f"\nИНФО О ПОИСКЕ:")
    print(f"Тип поиска: {search_result.search_type}")
    print(f"Найдено документов: {len(search_result.documents)}")
    
    code_count = len([d for d in search_result.documents if d.metadata.get('type') == 'code'])
    doc_count = len([d for d in search_result.documents if d.metadata.get('type') == 'doc'])
    print(f"Код: {code_count}, Документация: {doc_count}")
    
except Exception as e:
    print(f"Ошибка: {e}")

ОТВЕТ:
### Как использовать FAISS векторный магазин в LangChain?

FAISS (Fast Approximate Nearest Neighbor Search) — это популярная библиотека для приближенного поиска ближайших соседей в пространстве векторов. В LangChain эта библиотека может быть использована через соответствующий интеграционный модуль. Вот как это можно сделать:

#### Шаг 1: Установка необходимых пакетов
Для начала установим необходимые зависимости:

```bash
% pip install --upgrade --quiet  langchain-openai faiss-cpu
```

Это установит LangChain с поддержкой OpenAI API, а также необходимую версию библиотеки FAISS для CPU.

#### Шаг 2: Пример использования FAISS в LangChain

Рассмотрим простой пример, где мы загружаем предварительно обученные векторы и используем их для поиска ближайшего соседа.

```python
from langchain import LLM, Chain, FaissIndexer
import faiss

# Загрузим предварительно обученные векторы
vectors = faiss.read_index("path/to/your/embedding/file.index")

# Создадим индексатор на основе этих векторо

In [14]:
query = "How does similarity search work in FAISS?"

try:
    # Получаем ответ
    response = retrieval_chain.invoke({"input": query})
    
    # Выводим результат
    print("ОТВЕТ:")
    print(response["answer"])
    
    # Показываем информацию о поиске
    search_result = system.smart_search(query, k=3)
    print(f"\nИНФО О ПОИСКЕ:")
    print(f"Тип поиска: {search_result.search_type}")
    print(f"Найдено документов: {len(search_result.documents)}")
    
    code_count = len([d for d in search_result.documents if d.metadata.get('type') == 'code'])
    doc_count = len([d for d in search_result.documents if d.metadata.get('type') == 'doc'])
    print(f"Код: {code_count}, Документация: {doc_count}")
    
except Exception as e:
    print(f"Ошибка: {e}")

ОТВЕТ:
### Как работает поиск схожести в FAISS?

FAISS (Fast Approximate Nearest neigbor Search) — это высокопроизводительная библиотека для индексации и поиска ближайших соседей в пространстве признаков. Она поддерживает различные методы индексирования и позволяет эффективно искать похожие элементы в больших наборах данных.

#### 1. Принципы работы FAISS

FAISS использует подход k-d деревьев (k-мерных деревьев) для построения иерархической структуры, которая помогает быстро находить ближайшие точки в многомерном пространстве. Основные шаги поиска включают:

1. **Инициализация дерева**: Фактически, дерево строится как иерархическая структура, где каждый уровень разбивает пространство на два подпространства с помощью гиперплоскостей. Это позволяет быстро сужать область поиска.
   
2. **Вставка точек**: Каждая точка вставляется в соответствующее место в дереве. В зависимости от выбранного метода, может использоваться несколько вариантов разбиения пространства.

3. **Поиск ближайшего сосе

In [15]:
query = "What is a Document in LangChain and how to create it?"

try:
    # Получаем ответ
    response = retrieval_chain.invoke({"input": query})
    
    # Выводим результат
    print("ОТВЕТ:")
    print(response["answer"])
    
    # Показываем информацию о поиске
    search_result = system.smart_search(query, k=3)
    print(f"\nИНФО О ПОИСКЕ:")
    print(f"Тип поиска: {search_result.search_type}")
    print(f"Найдено документов: {len(search_result.documents)}")
    
    code_count = len([d for d in search_result.documents if d.metadata.get('type') == 'code'])
    doc_count = len([d for d in search_result.documents if d.metadata.get('type') == 'doc'])
    print(f"Код: {code_count}, Документация: {doc_count}")
    
except Exception as e:
    print(f"Ошибка: {e}")

ОТВЕТ:
Документ в LangChain представляет собой универсальный способ хранения неструктурированного текста вместе с его метаданными. Он используется как базовый тип данных в различных методах обработки естественного языка, таких как классификация текста, создание моделей с нуля и т. д. 

### Как создать документ в LangChain?

Документ в LangChain создается с помощью класса `Document`. Чтобы создать объект `Document`, необходимо передать два обязательных аргумента: текст (страницы контента) и словарь с метаданными. Вот как это выглядит на практике:

```python
from langchain import LLM, Document

# Создание объекта Document с текстом и метаданными
doc = Document(
    page_content="Это пример содержимого страницы",
    metadata={"author": "Иван Иванов", "topic": "Программирование"}
)
```

В данном примере:
- `page_content` — это строка, содержащая текст страницы.
- `metadata` — это словарь, который содержит дополнительные данные о документе, такие как автор, тема и любые другие поля, которы